## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# # 필요한 라이브러리를 설치합니다.
# !pip install timm
# !pip install matplotlib
# !pip install seaborn
# !pip install optuna
# !apt install -y libgl1-mesa-glx
# !pip install albumentations

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random
import copy

import optuna, math
import timm
import torch
import cv2
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler  # Mixed Precision용

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import wandb
from datetime import datetime

# 한글 폰트 설정 (시각화용)
plt.rcParams['font.family'] = ['DejaVu Sans']

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [4]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '../data/'

# model config
# model_name = 'tf_efficientnetv2_b3' # 'resnet50' 'efficientnet-b0', ...
# model_name = 'swin_base_patch4_window12_384_in22k'
model_name = 'convnext_large_384_in22ft1k'
# model_name = 'convnextv2_base.fcmae_ft_in22k_in1k_384'
# model_name = 'vit_base_patch16_clip_384.laion2b_ft_in12k_in1k' # openclip
# model_name = 'vit_base_patch16_384.augreg_in1k' # augreg
# model_name = 'eva02_enormous_patch14_plus_clip_224.laion2b_s9b_b144k' # eva-02 멀티모달
# model_name = 'eva02_large_patch14_448.mim_in22k_ft_in1k' #448 테스트용
# model_name = 'vit_base_patch14_reg4_dinov2.lvd142m' # dinov2 reg4

# model_name = 'eva02_large_patch14_448.mim_in22k_ft_in1k' #448 테스트용

# training config
img_size = 512
LR = 2e-4
EPOCHS = 10
BATCH_SIZE = 16
num_workers = 8
EMA = True  # Exponential Moving Average 사용 여부

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

# K-Fold 적용

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [5]:
# 5-Fold 앙상블 모델 준비
ensemble_models = []
for i in range(5):  # fold 개수만큼
    fold_model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)  # pretrained=False로 변경
    
    # fold별 저장된 파일 로드
    checkpoint = torch.load(f'models/fold_{i+1}_best.pth')  # fold별 파일 경로
    fold_model.load_state_dict(checkpoint)
    fold_model.eval()
    
    ensemble_models.append(fold_model)
    print(f"✓ Fold {i+1} model loaded from models/fold_{i+1}_best.pth")

print(f"Using ensemble of all {len(ensemble_models)} fold models for inference")

✓ Fold 1 model loaded from models/fold_1_best.pth
✓ Fold 2 model loaded from models/fold_2_best.pth
✓ Fold 3 model loaded from models/fold_3_best.pth
✓ Fold 4 model loaded from models/fold_4_best.pth
✓ Fold 5 model loaded from models/fold_5_best.pth
Using ensemble of all 5 fold models for inference


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [6]:
# Temperature Scaling 클래스 정의
class TemperatureScaling(nn.Module):
    def __init__(self):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1) * 1.5)
    
    def forward(self, logits):
        return logits / self.temperature

In [16]:
def get_classification_confidence(image_path, model):
    """분류 모델 신뢰도 기반 품질 측정"""
    try:
        # 이미지 로드 및 전처리
        img = cv2.imread(image_path)
        if img is None:
            return 0.0
        
        # RGB 변환
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 모델 입력용 transform 적용
        transform = A.Compose([
            A.LongestMaxSize(max_size=512),
            A.PadIfNeeded(min_height=512, min_width=512, border_mode=0, value=0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        
        # 이미지 전처리 및 배치 차원 추가
        processed = transform(image=img_rgb)['image'].unsqueeze(0).to(device)
        
        # 모델 예측 (첫 번째 앙상블 모델 사용)
        with torch.no_grad():
            logits = ensemble_models[0](processed)
            probs = torch.softmax(logits, dim=1)
            confidence = torch.max(probs).item()
        
        return confidence
        
    except Exception as e:
        print(f"신뢰도 측정 실패 ({image_path}): {e}")
        return 0.0

# 문서 특화 적응형 전처리 함수
def adaptive_preprocessing(image_path, quality_threshold=0.99):
    """문서 이미지 전문가 워크플로우 기반 전처리"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None, 0.0, 0.0
        
        # 분류 신뢰도 기반 품질 측정
        original_confidence = get_classification_confidence(image_path, ensemble_models[0])
        
        # 신뢰도가 높으면 전처리 스킵
        if original_confidence > quality_threshold:
        # if False: # 모든 이미지에 대해 전처리 시도
            return img, original_confidence, original_confidence
        
        # 1) 품질 진단
        processed_img = img.copy()
        gray = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
        
        g_std = gray.std()  # 대비
        lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()  # 블러
        mean_bright = gray.mean()  # 밝기
        
        # 스큐 각도 측정
        edges = cv2.Canny(gray, 50, 150)
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
        skew_angle = 0
        if lines is not None and len(lines) > 5:
            angles = []
            for line in lines[:20]:
                if len(line[0]) >= 2:
                    rho, theta = line[0]
                    angle = theta * 180 / np.pi - 90
                    if abs(angle) < 45:
                        angles.append(angle)
            if angles:
                skew_angle = abs(np.median(angles))
        
        # 2) 라우팅 결정
        needs_deskew = skew_angle >= 8
        low_contrast = g_std < 35
        is_blurry = 50 <= lap_var < 150
        very_blurry = lap_var < 50
        too_bright = mean_bright > 180
        has_noise = lap_var < 100
        
        # 3) 프리셋 선택
        if very_blurry and too_bright and has_noise:
            preset = "HEAVY"
        elif needs_deskew or skew_angle >= 5:
            preset = "MEDIUM"  
        else:
            preset = "LIGHT"
        
        # 4) 전처리 적용
        
        # 단계 0: 플립 감지 및 보정
        flipped = cv2.flip(processed_img, 1)
        gray_current = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
        gray_flipped = cv2.cvtColor(flipped, cv2.COLOR_BGR2GRAY)
        
        def get_text_density(image):
            binary = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 15, 10)
            return np.sum(binary == 0) / binary.size
        
        original_density = get_text_density(gray_current)
        flipped_density = get_text_density(gray_flipped)
        
        if flipped_density > original_density * 1.1:
            processed_img = flipped
        
        # 단계 1: Deskew (필요시)
        if needs_deskew and preset in ["MEDIUM", "HEAVY"]:
            h, w = processed_img.shape[:2]
            center = (w//2, h//2)
            
            valid_angles = []
            if lines is not None:
                for line in lines[:10]:
                    if len(line[0]) >= 2:
                        rho, theta = line[0]
                        angle = theta * 180 / np.pi - 90
                        if abs(angle) < 15:
                            valid_angles.append(angle)
            
            if len(valid_angles) > 0:
                rotation_angle = np.median(valid_angles)
                if abs(rotation_angle) >= 3:
                    M = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
                    processed_img = cv2.warpAffine(processed_img, M, (w, h), 
                                                 borderMode=cv2.BORDER_CONSTANT, 
                                                 borderValue=(255, 255, 255))
        
        # 단계 2: 밝기 조정 (HEAVY만)
        if too_bright and preset == "HEAVY":
            processed_img = np.power(processed_img/255.0, 0.92) * 255
            processed_img = processed_img.astype(np.uint8)
        
        # 단계 3: NLM 노이즈 제거 (HEAVY만)
        if has_noise and preset == "HEAVY":
            processed_img = cv2.fastNlMeansDenoisingColored(processed_img, None, 3, 3, 7, 21)
        
        # 단계 4: CLAHE (저대비인 경우)
        if low_contrast:
            lab = cv2.cvtColor(processed_img, cv2.COLOR_BGR2LAB)
            l_channel = lab[:, :, 0]
            clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
            l_channel = clahe.apply(l_channel)
            lab[:, :, 0] = l_channel
            processed_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        
        # 단계 5: Unsharp (블러인 경우)
        if is_blurry or very_blurry:
            gray = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
            gaussian = cv2.GaussianBlur(gray, (0, 0), 1.0)
            unsharp = cv2.addWeighted(gray, 1.4, gaussian, -0.4, 0)
            unsharp = np.clip(unsharp, 0, 255).astype(np.uint8)
            
            hsv = cv2.cvtColor(processed_img, cv2.COLOR_BGR2HSV)
            hsv[:, :, 2] = unsharp
            processed_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # 품질 검사
        temp_path = image_path.replace('.jpg', '_temp_processed.jpg')
        cv2.imwrite(temp_path, processed_img)
        processed_confidence = get_classification_confidence(temp_path, ensemble_models[0])
        os.remove(temp_path)

        if processed_confidence < original_confidence * 0.8:
            return img, original_confidence, original_confidence

        return processed_img, original_confidence, processed_confidence
        
    except Exception as e:
        print(f"전처리 실패 ({image_path}): {e}")
        img = cv2.imread(image_path)
        original_confidence = 0.5 if img is not None else 0.0
        return img, original_confidence, original_confidence

def assess_image_quality(image_path):
    """이미지 품질을 0-1 점수로 평가"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            return 0.0
            
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 블러 측정
        blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
        blur_normalized = min(blur_score / 1000.0, 1.0)
        
        # 대비 측정
        contrast_score = gray.std()
        contrast_normalized = min(contrast_score / 80.0, 1.0)
        
        # 밝기 분포 측정
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = hist.flatten()
        hist = hist[hist > 0]
        if len(hist) > 1:
            prob = hist / hist.sum()
            brightness_score = -np.sum(prob * np.log2(prob + 1e-8))
            brightness_normalized = brightness_score / 8.0
        else:
            brightness_normalized = 0.0
            
        quality_score = (
            0.5 * blur_normalized +
            0.3 * contrast_normalized +
            0.2 * brightness_normalized
        )
        
        return min(quality_score, 1.0)
        
    except Exception as e:
        return 0.0

print("전처리 함수 추가 완료!")

전처리 함수 추가 완료!


In [8]:
essential_tta_transforms = [
    # 원본
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 90도 회전들
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[90, 90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[180, 180], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.Rotate(limit=[-90, -90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 밝기 개선
    A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
        A.RandomBrightnessContrast(brightness_limit=[0.3, 0.3], contrast_limit=[0.3, 0.3], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
]

In [9]:
# TTA 추론을 위한 Dataset 클래스
class TTAImageDataset(Dataset):
    def __init__(self, data, path, transforms):
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
        self.path = path
        self.transforms = transforms  # 여러 transform을 리스트로 받음

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        #img = np.array(Image.open(os.path.join(self.path, name)).convert('RGB'))

        # 적응형 전처리 적용 (오염도 높은 이미지만 자동 선별)
        processed_img, original_confidence, processed_confidence = adaptive_preprocessing(img_path)

        # 전처리 적용 여부 판단
        preprocessing_applied = abs(processed_confidence - original_confidence) > 0.001

        # 통계 수집
        collect_preprocessing_stats(original_confidence, processed_confidence, preprocessing_applied)
        
        if processed_img is not None:
            # OpenCV BGR을 RGB로 변환
            img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
        else:
            # 전처리 실패 시 원본 사용
            img = np.array(Image.open(img_path).convert('RGB'))
        
        # 모든 transform을 적용한 결과를 리스트로 반환
        augmented_images = []
        for transform in self.transforms:
            aug_img = transform(image=img)['image']
            augmented_images.append(aug_img)
        
        return augmented_images, target

In [10]:
# TTA Dataset 생성
tta_dataset = TTAImageDataset(
    "../data/sample_submission.csv",
    "../data/test/",
    essential_tta_transforms
)

# TTA DataLoader (배치 크기를 줄여서 메모리 절약)
tta_loader = DataLoader(
    tta_dataset,
    batch_size=64,  # TTA는 메모리를 많이 사용하므로 배치 크기 줄임
    shuffle=False,
    # num_workers=num_workers,
    num_workers=0,
    pin_memory=True
)

print(f"TTA Dataset size: {len(tta_dataset)}")

TTA Dataset size: 3140


In [11]:
# 제출용
def ensemble_tta_inference(models, loader, transforms, confidence_threshold=0.9):
    """5-Fold 모델 앙상블 + TTA 추론"""
    temp_scaling = TemperatureScaling().to(device)  # Temperature Scaling 추가
    all_predictions = []
    
    for batch_idx, (images_list, _) in enumerate(tqdm(loader, desc="Ensemble TTA")):
        batch_size = images_list[0].size(0)
        
        ensemble_probs = torch.zeros(batch_size, 17).to(device)
                
        # 각 fold 모델별 예측
        for model in models:
            with torch.no_grad():
                # 각 TTA 변형별 예측
                for images in images_list:
                    images = images.to(device)
                    preds = model(images)
                    probs = torch.softmax(preds, dim=1)
                    
                    # Temperature Scaling 적용
                    scaled_preds = temp_scaling(preds)
                    probs = torch.softmax(scaled_preds, dim=1)
                    
                    ensemble_probs += probs / (len(models) * len(images_list))
        
        final_preds = torch.argmax(ensemble_probs, dim=1)
        all_predictions.extend(final_preds.cpu().numpy())
    
    return all_predictions

In [12]:
# 실험용: 단일 모델 TTA 추론 (5배 빠름)
def single_model_tta_inference(model, loader, transforms):
    """단일 모델 TTA 추론 - 실험용"""
    temp_scaling = TemperatureScaling().to(device)
    all_predictions = []
    
    for batch_idx, (images_list, _) in enumerate(tqdm(loader, desc="Single TTA")):
        batch_size = images_list[0].size(0)
        tta_probs = torch.zeros(batch_size, 17).to(device)
        
        # 단일 모델로 TTA
        with torch.no_grad():
            for images in images_list:
                images = images.to(device)
                preds = model(images)
                
                # Temperature Scaling 적용
                scaled_preds = temp_scaling(preds)
                probs = torch.softmax(scaled_preds, dim=1)
                
                tta_probs += probs / len(images_list)
        
        final_preds = torch.argmax(tta_probs, dim=1)
        all_predictions.extend(final_preds.cpu().numpy())
    
    return all_predictions

print("실험용 단일 추론 함수 추가 완료!")

실험용 단일 추론 함수 추가 완료!


In [13]:
# 전처리 통계 수집용 글로벌 변수 초기화
preprocessing_stats = {
    'total_images': 0,
    'preprocessing_applied': 0,
    'preprocessing_skipped': 0,
    'quality_improvements': [],
    'quality_degradations': []
}

def collect_preprocessing_stats(original_confidence, processed_confidence, applied):
    """전처리 통계 수집"""
    global preprocessing_stats
    preprocessing_stats['total_images'] += 1
    
    # 디버깅용 출력 (첫 5개만)
    if preprocessing_stats['total_images'] <= 5:
        print(f"Debug: 이미지 {preprocessing_stats['total_images']} 처리됨, 적용: {applied}")
    
    if applied:
        preprocessing_stats['preprocessing_applied'] += 1
        quality_change = processed_confidence - original_confidence
        if quality_change > 0:
            preprocessing_stats['quality_improvements'].append(quality_change)
        else:
            preprocessing_stats['quality_degradations'].append(quality_change)
    else:
        preprocessing_stats['preprocessing_skipped'] += 1

def print_preprocessing_stats():
    """전처리 통계 출력"""
    stats = preprocessing_stats
    total = stats['total_images']
    applied = stats['preprocessing_applied']
    skipped = stats['preprocessing_skipped']
    
    print(f"\n{'='*50}")
    print(f"전처리 통계 분석")
    print(f"{'='*50}")
    print(f"총 이미지 수: {total}개")
    
    # ZeroDivisionError 방지
    if total > 0:
        print(f"전처리 적용: {applied}개 ({applied/total*100:.1f}%)")
        print(f"전처리 스킵: {skipped}개 ({skipped/total*100:.1f}%)")
    else:
        print("전처리 적용: 0개 (0.0%)")
        print("전처리 스킵: 0개 (0.0%)")
        print("⚠️ 통계 수집이 작동하지 않았습니다.")
        return
    
    if stats['quality_improvements']:
        avg_improvement = np.mean(stats['quality_improvements'])
        print(f"품질 개선 평균: +{avg_improvement:.3f}")
        print(f"품질 개선 건수: {len(stats['quality_improvements'])}개")
    
    if stats['quality_degradations']:
        avg_degradation = np.mean(stats['quality_degradations'])
        print(f"품질 악화 평균: {avg_degradation:.3f}")
        print(f"품질 악화 건수: {len(stats['quality_degradations'])}개")

In [14]:
# GT 기반 전처리 효과 검증 함수 - 수정 버전
class CleanTTAImageDataset(Dataset):
    """전처리 없는 순수한 TTA Dataset"""
    def __init__(self, data, path, transforms):
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
        self.path = path
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        
        # 전처리 없이 원본 이미지만 로드
        img = np.array(Image.open(img_path).convert('RGB'))
        
        # 모든 transform을 적용한 결과를 리스트로 반환
        augmented_images = []
        for transform in self.transforms:
            aug_img = transform(image=img)['image']
            augmented_images.append(aug_img)
        
        return augmented_images, target

def evaluate_preprocessing_effect():
    """selected_images_500.csv GT로 전처리 전후 F1 스코어 비교 - 수정 버전"""
    
    global preprocessing_stats
    preprocessing_stats = {
        'total_images': 0,
        'preprocessing_applied': 0,
        'preprocessing_skipped': 0,
        'quality_improvements': [],
        'quality_degradations': []
    }
    
    # GT 파일 로드
    gt_file = "../data/selected_images_500.csv"
    try:
        gt_df = pd.read_csv(gt_file)
        gt_df = gt_df.rename(columns={'predicted_class': 'true_label'})
        print(f"GT 데이터 로드 완료: {len(gt_df)}개 이미지")
    except:
        print("selected_images_500.csv 파일을 찾을 수 없습니다.")
        return
    
    # 라벨링된 데이터만 필터링
    if 'true_label' in gt_df.columns:
        labeled_df = gt_df.dropna(subset=['true_label'])
        print(f"라벨링된 데이터: {len(labeled_df)}개")
    else:
        print("true_label 컬럼이 없습니다. GT 파일 형식을 확인하세요.")
        return
    
    gt_files = labeled_df['filename'].tolist()
    gt_labels = labeled_df['true_label'].astype(int).tolist()
    
    # 1) 전처리 OFF 추론 (CleanTTAImageDataset 사용)
    print("\n1단계: 전처리 없는 추론 (CleanTTAImageDataset)...")
    clean_dataset = CleanTTAImageDataset(
        labeled_df[['filename', 'true_label']].rename(columns={'true_label': 'target'}),
        "../data/test/",
        essential_tta_transforms
    )
    
    # clean_loader = DataLoader(clean_dataset, batch_size=32, shuffle=False, num_workers=4)
    clean_loader = DataLoader(clean_dataset, batch_size=32, shuffle=False, num_workers=0)
    
    single_model = ensemble_models[0]
    clean_preds = single_model_tta_inference(single_model, clean_loader, essential_tta_transforms)
    
    # 2) 전처리 ON 추론 (기존 TTAImageDataset 사용)  
    print("2단계: 전처리 적용된 추론 (TTAImageDataset)...")
    preprocessed_dataset = TTAImageDataset(
        labeled_df[['filename', 'true_label']].rename(columns={'true_label': 'target'}),
        "../data/test/",
        essential_tta_transforms
    )
    
    # preprocessed_loader = DataLoader(preprocessed_dataset, batch_size=32, shuffle=False, num_workers=4)
    preprocessed_loader = DataLoader(preprocessed_dataset, batch_size=32, shuffle=False, num_workers=0)
    preprocessed_preds = single_model_tta_inference(single_model, preprocessed_loader, essential_tta_transforms)
    
    # 3) F1 스코어 계산
    from sklearn.metrics import f1_score, classification_report
    
    f1_clean = f1_score(gt_labels, clean_preds, average='macro')
    f1_preprocessed = f1_score(gt_labels, preprocessed_preds, average='macro')
    f1_improvement = f1_preprocessed - f1_clean
    
    # 4) 결과 출력
    print("\n" + "="*50)
    print("전처리 효과 검증 결과 (수정 버전)")
    print("="*50)
    print(f"GT 데이터 수: {len(gt_labels)}개")
    print(f"전처리 전 F1 (Clean): {f1_clean:.4f}")
    print(f"전처리 후 F1 (Processed): {f1_preprocessed:.4f}")
    print(f"F1 개선: {f1_improvement:+.4f} ({f1_improvement/f1_clean*100:+.1f}%)")
    
    if f1_improvement > 0.01:
        print("✅ 전처리 효과 확인! 제출용 앙상블 모드 실행 권장")
    elif f1_improvement > 0.005:
        print("📊 미미한 개선. 추가 검토 필요")
    else:
        print("❌ 전처리 효과 미미. 설정 재검토 필요")
    
    # 5) 상세 분석
    unique_labels = sorted(set(gt_labels) | set(clean_preds) | set(preprocessed_preds))
    target_names = [f"Class_{i}" for i in unique_labels]

    print("\n전처리 전 (Clean Dataset):")
    print(classification_report(gt_labels, clean_preds, labels=unique_labels, target_names=target_names, digits=3))
    print("\n전처리 후 (Preprocessed Dataset):")
    print(classification_report(gt_labels, preprocessed_preds, labels=unique_labels, target_names=target_names, digits=3))
    
    # 전처리 통계 출력
    print_preprocessing_stats()
    
    return {
        'f1_clean': f1_clean,
        'f1_preprocessed': f1_preprocessed, 
        'improvement': f1_improvement,
        'gt_count': len(gt_labels)
    }

print("GT 기반 F1 비교 함수 수정 완료!")

GT 기반 F1 비교 함수 수정 완료!


In [ ]:
# ===========================================
# 실험 모드 설정 (실험용 vs 제출용)
# ===========================================
EXPERIMENT_MODE = True  # True: 단일모델, False: 5-fold 앙상블

if EXPERIMENT_MODE:
    print("🧪 실험 모드: 단일 모델 TTA 추론 (빠름)")
    # 첫 번째 모델만 사용
    single_model = ensemble_models[0]
    tta_predictions = single_model_tta_inference(
        model=single_model,
        loader=tta_loader, 
        transforms=essential_tta_transforms
    )
else:
    print("🚀 제출 모드: 5-Fold 앙상블 TTA 추론 (정확함)")
    tta_predictions = ensemble_tta_inference(
        models=ensemble_models, 
        loader=tta_loader, 
        transforms=essential_tta_transforms,
        confidence_threshold=0.9
    )

In [17]:
evaluation_results = evaluate_preprocessing_effect()

GT 데이터 로드 완료: 500개 이미지
라벨링된 데이터: 500개

1단계: 전처리 없는 추론 (CleanTTAImageDataset)...


Single TTA: 100%|██████████| 16/16 [01:15<00:00,  4.73s/it]


2단계: 전처리 적용된 추론 (TTAImageDataset)...


Single TTA:   0%|          | 0/16 [00:00<?, ?it/s]

Debug: 이미지 1 처리됨, 적용: True
Debug: 이미지 2 처리됨, 적용: True
Debug: 이미지 3 처리됨, 적용: False
Debug: 이미지 4 처리됨, 적용: True
Debug: 이미지 5 처리됨, 적용: True


Single TTA: 100%|██████████| 16/16 [01:55<00:00,  7.22s/it]


전처리 효과 검증 결과 (수정 버전)
GT 데이터 수: 500개
전처리 전 F1 (Clean): 0.6796
전처리 후 F1 (Processed): 0.5458
F1 개선: -0.1338 (-19.7%)
❌ 전처리 효과 미미. 설정 재검토 필요

전처리 전 (Clean Dataset):
              precision    recall  f1-score   support

     Class_3      0.686     0.901     0.779       121
     Class_4      0.985     0.832     0.902       161
     Class_7      0.838     0.763     0.799       169
    Class_10      0.000     0.000     0.000         0
    Class_13      0.000     0.000     0.000         0
    Class_14      0.918     0.918     0.918        49

   micro avg      0.834     0.834     0.834       500
   macro avg      0.571     0.569     0.566       500
weighted avg      0.856     0.834     0.839       500


전처리 후 (Preprocessed Dataset):
              precision    recall  f1-score   support

     Class_3      0.671     0.893     0.766       121
     Class_4      0.925     0.845     0.883       161
     Class_7      0.845     0.710     0.772       169
    Class_10      0.000     0.000     0.000    

In [17]:
# TTA 결과로 submission 파일 생성
tta_pred_df = pd.DataFrame(tta_dataset.df, columns=['ID', 'target'])
tta_pred_df['target'] = tta_predictions

In [18]:
# 기존 submission과 동일한 순서인지 확인
sample_submission_df = pd.read_csv("../data/sample_submission.csv")
assert (sample_submission_df['ID'] == tta_pred_df['ID']).all()

In [19]:
# TTA 결과 저장
tta_pred_df.to_csv("../submission/choice.csv", index=False)
print("TTA predictions saved")

print("TTA Prediction sample:")

TTA predictions saved
TTA Prediction sample:


In [20]:
tta_pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
